In [3]:
import numpy as np
import scipy
import pylab as plt
from sklearn import linear_model
%matplotlib inline
from scipy import stats

# Datum
trainX = np.loadtxt('X_train.csv', delimiter=',', skiprows=1)
traint = np.loadtxt('y_train.csv', delimiter=',', skiprows=1)
traint = traint[:,1][:,None]

def random_classifier():
    return np.random.randint(1,2+1)

def cv(trainX, traint, folds=10, K=2):
    slice_size = int(trainX.shape[0]/folds)
    total_accuracy = 0
    
    for i in range(folds):
        split_start = slice_size * i
        split_end = slice_size * (i+1)
        cv_training_X = np.concatenate((trainX[:split_start], trainX[split_end:]))
        cv_training_t = np.concatenate((traint[:split_start], traint[split_end:]))
        cv_testing_X = trainX[split_start:split_end]
        cv_testing_t = traint[split_start:split_end]
        
        # Logit
        w_nr = get_nr_weights(cv_training_X, cv_training_t)
        #print("Weights for logit:")
        #print(w_nr)
        
        hits = 0
        for j in range(slice_size):
            
            #
            # Pick classifier here
            #
            #c = knn_classifier(cv_training_X, cv_training_t, cv_testing_X[j,:], K)
            c = logit_classifier(w_nr, cv_testing_X[j,:])
            #c = random_classifier()
            
            if (c == cv_testing_t[j]):
                hits += 1
        accuracy = hits/cv_testing_t.shape[0]
        #print("Iteration and accuracy:", i, accuracy)
        total_accuracy += accuracy
    
    total_accuracy /= folds
    return total_accuracy
    

"""def knn_classifier(trainX, traint, testrow, K=2):
    distances = ((trainX - testrow)**2).sum(axis=1)
    dc = zip(distances,traint)
    dc = sorted(dc,key = lambda x:x[0])
    classes = []
    votes = []
    for k in range(K):
        this_class = dc[k][1] # get the class of the kth ranked
        if not this_class in classes:
            classes.append(this_class)
            votes.append(1)
        else:
            index = classes.index(this_class)
            votes[index] += 1
    best_class = classes[0]
    best_vote = votes[0]
    pos = 1
    for cl in classes[1:]:
        if votes[pos] > best_vote:
            best_vote = votes[pos]
            best_class = cl
        pos += 1
    return best_class"""

"""def main_knn(K=100):
    Ks = np.arange(1,K,1)
    for k in Ks:
        accuracy = cv(trainX, traint, 20, k)
        print("K and accuracy:", k, accuracy)"""
        
def get_nr_weights(trainx, traint, n_itrs=20):
    n_weights = trainx.shape[1]
    m = trainx.shape[0]
    lamb = 0.001
    
    w = np.zeros((n_weights,1))
    dx = np.diag(np.dot(trainx, trainx.T))[:,None]
    allw = np.zeros((n_weights,n_itrs))

    for i in range(n_itrs):
        allw[:,i] = w.flatten()
        #P = 1.0/(1.0 + np.exp(-np.dot(trainx, w)))
        P = sigmoid(np.dot(trainx, w))
        
        gw = -w + np.sum(trainx*np.tile(traint-P,(1,n_weights)), axis=0)[:,None]
        temp = trainx*np.tile(P*(1-P), (1,n_weights))
        hw = -np.eye(n_weights) - np.dot(temp.T, trainx)
        w = w - np.dot(np.linalg.inv(hw), gw)
    
    #plt.plot(allw[0,:])
    #plt.plot(allw[1,:])
    #plt.xlabel('Iteration')
    #plt.ylabel('w')
    
    return w

#def sigmoid(testrow, w):
def sigmoid(x):
    #return 1.0/(1.0 + np.exp(-np.dot(testrow, w)))
    return scipy.special.expit(x)

def logit_classifier(w, testrow):
    x = np.dot(testrow, w)
    c = sigmoid(x)
    return c

def filter_by_distinction(trainX, traint, n=10):
    ps = np.zeros(trainX.shape[1])
    
    for i in range(trainX.shape[1]):
        feature = trainX.T[i][:,None]
        label = traint.flatten()[:,None]
        
        stacked = np.hstack((feature, label))
        stacked.sort(axis=0)
        count = (stacked[:,1] == 1).sum()
        group1 = stacked[:count,0]
        group2 = stacked[count:,0]
        
        p = stats.ttest_ind(group1,group2)[1]
        ps[i] = p
        
    features_by_distinction = ps.argsort()[:n]
    newX = np.zeros((trainX.shape[0], n))
    
    for i in range(n):
        newX[:,i] = trainX[:,features_by_distinction[i]]
    
    return newX
    

"""for fc in range(0, 100, 1):
    filtered_trainX = filter_by_distinction(trainX, traint, fc)
    score = cv(filtered_trainX, traint, folds=20, K=10)
    #print("Feature count and KNN accuracy:", fc, score)"""

# Logit
#filteredX = filter_by_distinction(trainX, traint, 2)
filteredX = trainX
#accuracy = cv(filteredX, traint, 10)

#print("Accuracy:", accuracy)

def sklearn_classifier(trainX, traint, testrows):
    """logreg = linear_model.LogisticRegression(penalty="l2",
                                            dual=False,
                                            tol=0.0001,
                                            C=1.0,
                                            fit_intercept=True,
                                            intercept_scaling=1,
                                            class_weight=None,
                                            random_state=None,
                                            solver="liblinear",
                                            max_iter=100,
                                            multi_class="ovr",
                                            verbose=0,
                                            warm_start=False,
                                            n_jobs=1)"""
    logreg = linear_model.LogisticRegression()
    
    logreg.fit(trainX, traint)
    pred = logreg.predict(testrows)
    return pred



#a = sklearn_classifier(filteredX, traint, filteredX[10])
#print(a)




# Submit, lol
testX = np.loadtxt('X_test.csv', delimiter=',', skiprows=1)
N = testX.shape[0]

#output = np.zeros((N, 2), dtype=int)
preds = sklearn_classifier(filteredX, traint.flatten(), testX)

output = np.ones((N, 2))
output[:,0] = range(N)
output[:,1] = preds

"""for i in range(N):
    c = knn_classifier(trainX, traint, testX[i,:], K=10)
    output[i,1] = c"""

np.savetxt('pred_classification.csv', output, fmt='%d', delimiter=",", header="Id,EpiOrStroma")